# Données des objets perdus

In [18]:
import requests

In [19]:
def specific_url_lostitem(annee):
    URL = "https://ressources.data.sncf.com/"
    ressource = "api/records/1.0/search/?dataset=objets-trouves-restitution&q=&rows=10000&sort=date&refine.gc_obo_gare_origine_r_name=Lille+Europe&refine.date="
    month_format = f"{annee}"
    return URL + ressource + month_format


In [22]:
from model import LostItem
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

engine = create_engine('sqlite:///db.sqlite')
Session = sessionmaker(bind=engine)
session = Session()

field_list = [
    ["type_objet", "gc_obo_type_c"],
    ["date_restitution", "gc_obo_date_heure_restitution_c"]
]

for annee in [2016,2017,2018,2019,2020,2021,2022]:
    my_request = requests.get(specific_url_lostitem(annee))
    print(f"request réalisée pour {annee}")
    for lost_item in my_request.json()["records"]:   
        lost_item_data= {}
        
        for field in field_list:    
            try: 
                lost_item_data[field[0]] = lost_item["fields"][field[1]]
            except KeyError:
                lost_item_data[field[0]]=None

        session.add(LostItem(date=lost_item["fields"]["date"],**lost_item_data))
    
    session.commit()

request réalisée pour 2016


/var/folders/py/4rt338cj1ks4z3332m7l3f880000gp/T/ipykernel_26446/3245594921.py:28: SAWarning: Column 'LostItem.id' is marked as a member of the primary key for table 'LostItem', but has no Python-side or server-side default generator indicated, nor does it indicate 'autoincrement=True' or 'nullable=True', and no explicit value is passed.  Primary key columns typically may not store NULL. Note that as of SQLAlchemy 1.1, 'autoincrement=True' must be indicated explicitly for composite (e.g. multicolumn) primary keys if AUTO_INCREMENT/SERIAL/IDENTITY behavior is expected for one of the columns in the primary key. CREATE TABLE statements are impacted by this change as well on most backends.
  session.commit()


request réalisée pour 2017


/var/folders/py/4rt338cj1ks4z3332m7l3f880000gp/T/ipykernel_26446/3245594921.py:28: SAWarning: Identity map already had an identity for (<class 'model.LostItem'>, (None, '2017-12-08T08:37:20+00:00'), None), replacing it with newly flushed object.   Are there load operations occurring inside of an event handler within the flush?
  session.commit()
/var/folders/py/4rt338cj1ks4z3332m7l3f880000gp/T/ipykernel_26446/3245594921.py:28: SAWarning: Identity map already had an identity for (<class 'model.LostItem'>, (None, '2017-12-09T08:10:34+00:00'), None), replacing it with newly flushed object.   Are there load operations occurring inside of an event handler within the flush?
  session.commit()
/var/folders/py/4rt338cj1ks4z3332m7l3f880000gp/T/ipykernel_26446/3245594921.py:28: SAWarning: Identity map already had an identity for (<class 'model.LostItem'>, (None, '2017-07-07T09:40:21+00:00'), None), replacing it with newly flushed object.   Are there load operations occurring inside of an event h

request réalisée pour 2018
request réalisée pour 2019
request réalisée pour 2020
request réalisée pour 2021
request réalisée pour 2022


In [38]:
import dateparser
print(dateparser.parse("last week"))

2022-12-26 15:04:34.129774


In [32]:
def request_lost_item(start_date,end_date):
    if isinstance(start_date,str): 
        import dateparser
        start_date = dateparser.parse(start_date)
    if isinstance(end_date,str): 
        import dateparser
        end_date = dateparser.parse(end_date)
    
    start_date = str(start_date.date())
    end_date =  str(end_date.date())
    URL = "https://ressources.data.sncf.com/"
    ressource = f"api/records/1.0/search/?dataset=objets-trouves-restitution&q=date:[{start_date}%20TO%20{end_date}]&rows=10000&sort=date&refine.gc_obo_gare_origine_r_name=Lille+Europe"
    
    my_request = requests.get(URL + ressource)

    for lost_item in my_request.json()["records"]:   
        lost_item_data= {}
        
        for field in field_list:    
            try: 
                lost_item_data[field[0]] = lost_item["fields"][field[1]]
            except KeyError:
                lost_item_data[field[0]]=None

        session.add(LostItem(date=lost_item["fields"]["date"],**lost_item_data))
    
    session.commit()

    return URL + ressource
    

    


In [34]:
from datetime import datetime
request_lost_item("last week", datetime.now())

'https://ressources.data.sncf.com/api/records/1.0/search/?dataset=objets-trouves-restitution&q=date:[2022-12-26%20TO%202023-01-02]&rows=10000&sort=date&refine.gc_obo_gare_origine_r_name=Lille+Europe'

In [10]:
from datetime import datetime
specific_url_lostitem("today", datetime.now())

datetime.datetime